In [1]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg/polars-0.20.16-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import polars as pl
import duckdb as dd
import os
from datetime import datetime
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

import tensorflow as tf
# import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1730550459.243532      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1102 12:27:39.251699160      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1102 12:27:39.251713339      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1102 12:27:39.251716594      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1102 12:27:39.251718968      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1102 12:27:39.251721437      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [3]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1730550491.950694      13 service.cc:145] XLA service 0x5ca80a283fd0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730550491.950749      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1730550491.950754      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1730550491.950757      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1730550491.950761      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1730550491.950763      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1730550491.950766      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1730550491.950776      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1730550491.950779      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 96


In [6]:
Test = True

if Test:
    num_jobs = 92
    config_dict = {'csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'}
    config_dict['parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/'
else:
    num_jobs = 4
    config_dict = {'csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'}
    config_dict['parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/'

In [7]:
train_data = pl.read_csv(config_dict['csv_path'], low_memory=True)
# train_data.describe()
train_data_id_list = pl.Series(train_data.select(pl.col('id'))).to_list()
# len(train_data_id_list)

# actigraphy_file_list = []
actigraphy_id_list = []

for dirname, _, filenames in os.walk(config_dict['parquet_root_path']):
    for filename in filenames:
        full_parquet_file_path = os.path.join(dirname, filename)
        # actigraphy_file_list.append(full_parquet_file_path)
        actigraphy_id_list.append(full_parquet_file_path.split('/')[-2].replace('id=',''))
        
final_id_list = list(set(actigraphy_id_list).intersection(train_data_id_list))

print(len(final_id_list))
print(len(train_data_id_list))

996
3960


In [8]:
categorical_colms = ['Basic_Demos-Enroll_Season','CGAS-Season','Physical-Season','Fitness_Endurance-Season','FGC-Season'\
                    ,'BIA-Season','PCIAT-Season','PreInt_EduHx-Season','PAQ_A-Season','PAQ_C-Season','SDS-Season']

for cat_col in categorical_colms:
    cat_vals_list = pl.Series(train_data.select(pl.col(cat_col))).to_list()
    encoded_vals_list = le.fit_transform(cat_vals_list)
    encoded_col = "encoded_{0}".format(cat_col)
    train_data = train_data.with_columns(pl.Series(name=encoded_col, values=encoded_vals_list))
    train_data = train_data.drop([cat_col])

In [9]:
# Calculate the approximate size of each smaller list
chunk_size = len(train_data_id_list) // num_jobs
remainder = len(train_data_id_list) % num_jobs

# Create the num_jobs lists
result = []
start = 0

for i in range(num_jobs):
    # Add an extra element to the first few lists to account for remainder
    end = start + chunk_size + (1 if i < remainder else 0)
    result.append(train_data_id_list[start:end])
    start = end

# Check the result
"""for i, sublist in enumerate(result):
    print(f"List {i + 1} (length {len(sublist)}): {sublist}")"""

'for i, sublist in enumerate(result):\n    print(f"List {i + 1} (length {len(sublist)}): {sublist}")'

In [10]:
def get_hours(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[0])

def get_mins(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[1])
    
def get_secs(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[2])

In [11]:
def transform_parquet_file(parquet_file_path, file_id):

    directory = Path(parquet_file_path)

    # Check if any Parquet file exists in the directory
    parquet_files = list(directory.glob("*.parquet"))

    if parquet_files:
        parquet_df = pl.read_parquet(parquet_file_path+"*.parquet")
        parquet_df = parquet_df.drop(['id'])
        parquet_df = parquet_df.with_columns([
            pl.lit(file_id).alias("id"),
            pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day")
        ])

        parquet_df = parquet_df.drop(['time_of_day'])
        parquet_df = parquet_df.group_by(["id","weekday","hour_of_day","relative_date_PCIAT"])\
        .agg([pl.mean("enmo").name.suffix("_avg"), pl.mean("anglez").name.suffix("_avg"), pl.mean("light").name.suffix("_avg")\
              , pl.mean("battery_voltage").name.suffix("_avg")])
    else:
        columns = [("id", pl.String), ("weekday", pl.Int8), ("hour_of_day", pl.Int32), ("relative_date_PCIAT", pl.Float32)\
                  , ("enmo_avg", pl.Float32), ("anglez_avg", pl.Float32), ("light_avg", pl.Float32)\
                   , ("battery_voltage_avg", pl.Float32)]

        parquet_df = pl.DataFrame(schema=columns)
    
    return parquet_df

In [12]:
def unify_files(list_of_ids):
    df_initializing_id = list_of_ids.pop()
    parquet_file_name = "id={0}/".format(df_initializing_id)
    first_parquet_file = config_dict['parquet_root_path']+parquet_file_name
    first_parquet_df = transform_parquet_file(first_parquet_file, df_initializing_id)
    
    train_data_of_id = train_data.filter(pl.col('id')==df_initializing_id)
    first_joined_features_df = train_data_of_id.join(first_parquet_df, on="id", how="left")
    
    for id in tqdm(list_of_ids):
        parquet_file_name = "id={0}/".format(id)
        parquet_file_path = config_dict['parquet_root_path']+parquet_file_name
        parquet_df = transform_parquet_file(parquet_file_path, id)
        
        train_data_of_id = train_data.filter(pl.col('id')==id)
        joined_features = train_data_of_id.join(parquet_df, on="id", how="left")
        
        try:
            first_joined_features_df = pl.concat([first_joined_features_df, joined_features])
        except:
            print(id)
    return first_joined_features_df

In [13]:
parts_of_parquet = (Parallel(n_jobs=num_jobs)(delayed(unify_files)(id_list) for id_list in result))

100%|██████████| 42/42 [00:59<00:00,  1.41s/it]


In [16]:
test_df = parts_of_parquet[0]
#dd.sql("select * from test_df limit 100").pl()

In [18]:
from sklearn.preprocessing import StandardScaler

test_df_pandas = test_df.to_pandas()
#print(test_df_pandas["Physical-BMI"].isna().any())
complete_column = "id"

missing_mask = test_df_pandas.drop(columns=[complete_column]).isna()

for col in missing_mask.columns:
    test_df_pandas[col] = test_df_pandas[col].fillna(test_df_pandas[col].mean())

#test_df_pandas.head(100)

True


In [19]:
import pandas as pd
scaler = StandardScaler()

df_scaled_values = scaler.fit_transform(test_df_pandas.drop(columns=[complete_column]))
df_scaled = pd.DataFrame(df_scaled_values, columns=test_df_pandas.drop(columns=[complete_column]).columns)

# Add the complete column back to the scaled DataFrame
df_scaled[complete_column] = test_df_pandas[complete_column].values
df_scaled = df_scaled[test_df_pandas.columns]  # Ensure original column order

#df_scaled.head(100)

In [24]:
# Parameters for the MICE-like imputation
iterations = 17  # Number of iterations for imputation
epochs = 35  # Training epochs for each imputer model
BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync  # Batch size for training


# Function to create a simple neural network model for imputation
def create_imputer_model(input_dim):
    df_input = tf.keras.layers.Input(shape=(input_dim,), name="df_input")
    hidden_layer1 = tf.keras.layers.Dense(85, activation="relu")(df_input)
    hidden_layer2 = tf.keras.layers.Dense(65, activation="relu")(hidden_layer1)
    hidden_layer3 = tf.keras.layers.Dense(35, activation="relu")(hidden_layer2)
    df_output = tf.keras.layers.Dense(1)(hidden_layer3)
    
    model = tf.keras.Model(df_input, df_output)
    model.compile(optimizer="adamax", loss="mean_squared_error")
    return model

In [36]:
import numpy as np

col = 'Physical-BMI'
missing_rows = missing_mask[col]

if not missing_rows.any():
    print("no missing data; skipping over")
else:
    X_train = df_scaled.loc[~missing_rows].drop(columns=[col, complete_column]).values
    y_train = df_scaled.loc[~missing_rows, col].values
    X_pred = df_scaled.loc[missing_rows].drop(columns=[col, complete_column]).values

    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_ds = train_ds.batch(batch_size=BATCH_SIZE, drop_remainder=False)
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
    
    pred_ds = tf.data.Dataset.from_tensor_slices(X_pred)
    pred_ds = pred_ds.batch(batch_size=BATCH_SIZE, drop_remainder=False)
    pred_ds = pred_ds.prefetch(tf.data.AUTOTUNE)

In [27]:
with tpu_strategy.scope():
    imputer_model = create_imputer_model(input_dim=X_train.shape[1])
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    imputer_model.fit(train_ds, epochs=epochs, callbacks=[early_stopping_cb])

Epoch 1/35


2024-11-02 12:37:39.934388: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1730551060.007023     923 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(ff3c6585fb93079b:0:0), session_name()


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.3726
Epoch 2/35


I0000 00:00:1730551060.361463     923 tpu_compile_op_common.cc:245] Compilation of ff3c6585fb93079b:0:0 with session name  took 354.359194ms and succeeded
I0000 00:00:1730551060.365484     923 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(ff3c6585fb93079b:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_12586302780582042046", property.function_library_fingerprint = 8114739729407744959, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1730551060.365516     923 tpu_compilation_cache_interface.cc:541] After adding entry for key ff3c6585fb93079b:0:

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1065 
Epoch 3/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0541 
Epoch 4/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0500 
Epoch 5/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0593 
Epoch 6/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0530 
Epoch 7/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0374 
Epoch 8/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0194 
Epoch 9/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0147  
Epoch 10/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0120 
Epoch 11/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0103 
Epoch 12/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0089 
Epoch 13/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0083 
Epoch 14/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0074 
Epoch 15/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0068 
Epoch 16/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0065 
Epoch 17/35
2

In [28]:
with tpu_strategy.scope():
    predicted_values = imputer_model.predict(pred_ds)

df_scaled.loc[missing_rows, col] = predicted_values

3/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step 

I0000 00:00:1730551102.323606     921 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(46b927f6009dca76:0:0), session_name()
I0000 00:00:1730551102.381366     921 tpu_compile_op_common.cc:245] Compilation of 46b927f6009dca76:0:0 with session name  took 57.725163ms and succeeded
I0000 00:00:1730551102.381725     921 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(46b927f6009dca76:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_3001699512433967351", property.function_library_fingerprint = 13391374956198879495, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "", property.guaranteed_constants_size = 0, embedding_partit

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step


I0000 00:00:1730551102.743559     920 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(d23a5895cd15e6bf:0:0), session_name()
I0000 00:00:1730551102.814819     920 tpu_compile_op_common.cc:245] Compilation of d23a5895cd15e6bf:0:0 with session name  took 71.223258ms and succeeded
I0000 00:00:1730551102.815377     920 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(d23a5895cd15e6bf:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_4749942726664743389", property.function_library_fingerprint = 13578819203757294200, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,87,;", property.guaranteed_constants_size = 0, embedding_

In [29]:
df_scaled[test_df_pandas.drop(columns=[complete_column]).columns] \
= scaler.inverse_transform(df_scaled[test_df_pandas.drop(columns=[complete_column]).columns])

df_imputed = df_scaled

In [30]:
df_imputed

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,encoded_PAQ_A-Season,encoded_PAQ_C-Season,encoded_SDS-Season,weekday,hour_of_day,relative_date_PCIAT,enmo_avg,anglez_avg,light_avg,battery_voltage_avg
0,023eba7b,16.0,0.0,61.940529,21.937701,66.500000,138.000000,31.000000,70.000000,99.000000,...,3.0,4.0,4.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
1,00008ff9,5.0,0.0,51.000000,16.877316,46.000000,50.800000,26.846154,63.538413,82.602194,...,4.0,4.0,4.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
2,000fd460,9.0,0.0,61.940529,14.035590,48.000000,46.000000,22.000000,75.000000,70.000000,...,4.0,0.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
3,00105258,10.0,1.0,71.000000,16.648696,56.500000,75.600000,26.846154,65.000000,94.000000,...,4.0,2.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
4,00115b9f,9.0,0.0,71.000000,18.292347,56.000000,81.600000,26.846154,60.000000,97.000000,...,4.0,3.0,2.0,6.000000,5.000000,43.000000,0.044082,-14.720180,14.921977,4139.472656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,0219f64a,6.0,0.0,68.000000,16.377681,48.500000,54.800000,24.000000,67.000000,81.000000,...,4.0,4.0,1.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
3200,021b2020,9.0,1.0,61.940529,20.631322,53.500000,84.000000,28.000000,64.000000,81.000000,...,4.0,3.0,4.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
3201,0232fd98,14.0,1.0,61.940529,22.860295,59.866617,113.389711,26.846154,63.538413,82.602194,...,4.0,4.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475
3202,02347f13,10.0,0.0,61.940529,20.033783,59.000000,99.200000,28.000000,79.000000,88.000000,...,4.0,0.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368334,28.082163,3868.136475


In [33]:
df_imputed["Physical-BMI"].isna().any()
#df_imputed.isna().any()

False

In [35]:
len(train_ds)

0

In [37]:
with tpu_strategy.scope():
    for iteration in range(iterations):
        print(f"\nIteration {iteration + 1}/{iterations}")

        # Iterate over each column to impute, excluding the complete column
        for col in df_scaled.columns:
            print("Starting with column --> {0}".format(col))

            if col == complete_column:
                print("complete column; skipping over")
                continue  # Skip the complete column

            # Use the original missing mask to find rows with missing values
            missing_rows = missing_mask[col]

            if not missing_rows.any():
                print("no missing data; skipping over")
                continue
            
            X_train = df_scaled.loc[~missing_rows].drop(columns=[col, complete_column]).values
            y_train = df_scaled.loc[~missing_rows, col].values
            X_pred = df_scaled.loc[missing_rows].drop(columns=[col, complete_column]).values

            train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
            train_ds = train_ds.batch(batch_size=BATCH_SIZE, drop_remainder=False)
            train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

            pred_ds = tf.data.Dataset.from_tensor_slices(X_pred)
            pred_ds = pred_ds.batch(batch_size=BATCH_SIZE, drop_remainder=False)
            pred_ds = pred_ds.prefetch(tf.data.AUTOTUNE)

            # Create and train the imputer model
            imputer_model = create_imputer_model(input_dim=X_train.shape[1])
            early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
            imputer_model.fit(train_ds, epochs=epochs, callbacks=[early_stopping_cb], verbose=1)

            # Predict missing values and update the DataFrame
            predicted_values = imputer_model.predict(pred_ds)
            df_scaled.loc[missing_rows, col] = predicted_values
            
    df_scaled[test_df_pandas.drop(columns=[complete_column]).columns] \
    = scaler.inverse_transform(df_scaled[test_df_pandas.drop(columns=[complete_column]).columns])

    df_imputed = df_scaled


Iteration 1/17
Starting with column --> id
complete column; skipping over
Starting with column --> Basic_Demos-Age
no missing data; skipping over
Starting with column --> Basic_Demos-Sex
no missing data; skipping over
Starting with column --> CGAS-CGAS_Score
Epoch 1/35


2024-11-02 12:53:18.467343: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1730551998.682613     927 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(9723a1e20215fb4:0:0), session_name()


19/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 100770.7969

I0000 00:00:1730551999.024459     927 tpu_compile_op_common.cc:245] Compilation of 9723a1e20215fb4:0:0 with session name  took 341.798434ms and succeeded
I0000 00:00:1730551999.026362     927 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(9723a1e20215fb4:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_13208661830236348446", property.function_library_fingerprint = 4148866326977917458, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "16,87,;16,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1730551999.026388     927 tpu_compilation_cache_interface.cc:541] After adding entry for key 9723a1e202

25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 83648.2891
Epoch 2/35


...

I0000 00:00:1730553076.323621     895 tpu_compile_op_common.cc:245] Compilation of cdfe79448a8e650f:0:0 with session name  took 334.447562ms and succeeded
I0000 00:00:1730553076.325822     895 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(cdfe79448a8e650f:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_17992558739060785493", property.function_library_fingerprint = 13554614761942018341, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "2,87,;2,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1730553076.325849     895 tpu_compilation_cache_interface.cc:541] After adding entry for key cdfe79448

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 521ms/step - loss: 62252.9141
Epoch 3/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step - loss: 16961.9941
Epoch 4/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step - loss: 1251.6772
Epoch 5/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step - loss: 899.6382
Epoch 6/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step - loss: 5947.5654
Epoch 7/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - loss: 9802.5361
Epoch 8/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 448ms/step - loss: 11919.4648
Epoch 9/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 445ms/step - loss: 11758.7412
Epoch 10/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step - loss: 10050.7412
Epoch 11/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step - loss: 7684.2241
Epoch 12/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step - loss: 4927.4365
Epoch 13/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step - loss: 2769.1851
Epoch 14/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step - loss: 1259.8955
Epoch 15/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step - loss: 393.2402
Epoch 16/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step

2024-11-02 13:11:33.490588: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node functional_49_1/dense_196_1/Add/ReadVariableOp.
I0000 00:00:1730553093.517799     934 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(670d0357d387d579:0:0), session_name()
I0000 00:00:1730553093.574291     934 tpu_compile_op_common.cc:245] Compilation of 670d0357d387d579:0:0 with session name  took 56.448853ms and succeeded
I0000 00:00:1730553093.574920     934 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(670d0357d387d579:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_16787825641443664690", property.function_library_fingerprint = 394275096332254822, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 

23/25 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

TypeError: '>' not supported between instances of 'int' and 'NoneType'